# Pruebas realizadas sobre un nanotubo

In [1]:
from pylab import *
from ase.build import nanotube
from ase.build import graphene_nanoribbon
import plotly.graph_objects as go
from numpy import linalg as LA
from scipy.linalg import ishermitian

In [2]:
cnt1 = nanotube(4,2, length=1)
#cnt1     = graphene_nanoribbon(10,2, type='zigzag')
r        = cnt1.get_positions()
X,Y,Z    = cnt1.get_positions().T
nsites   = len(X)
a1,a2,a3 = array(cnt1.get_cell())

masas = cnt1.get_masses()

print(f"nsites = {nsites}")
print(f"    a1 = {a1}")
print(f"    a2 = {a2}")
print(f"    a3 = {a3}")

nsites = 56
    a1 = [0. 0. 0.]
    a2 = [0. 0. 0.]
    a3 = [ 0.          0.         11.27090059]


In [3]:
def findnn(X,Y,Z):
  nn = {}
  for todos in range(len(X)):

      UnSitio     = array([X[todos],Y[todos],Z[todos]])
      DataUnSitio = kron(ones(nsites),UnSitio)
      DataUnSitio = DataUnSitio.reshape(nsites,3)

      Dif = DataUnSitio - array([X,Y,Z]).T
      normas = []
      for n in range(len(Dif)):
          norm = sqrt(dot(Dif[n],Dif[n]))
          normas.append(norm)
      normas  = array(normas)
      logical = normas<1.44 # distancia en Ang para encontrar los vecinos

      nn[todos] = delete( logical.nonzero()[0], \
          where(logical.nonzero()[0] == todos) )
  return nn

def findnn_minus(X,Y,Z):
  nn_minus_a3 = {}
  for todos in range(len(X)):

      UnSitio     = array([X[todos]+a3[0],Y[todos]+a3[1],Z[todos]+a3[2]])
      DataUnSitio = kron(ones(nsites),UnSitio)
      DataUnSitio = DataUnSitio.reshape(nsites,3)

      Dif = DataUnSitio - array([X,Y,Z]).T
      normas = []
      for n in range(len(Dif)):
          norm = sqrt(dot(Dif[n],Dif[n]))
          normas.append(norm)
      normas  = array(normas)
      logical = normas<1.44 # distancia en Ang para encontrar los vecinos

      nn_minus_a3[todos] = delete( logical.nonzero()[0], \
          where(logical.nonzero()[0] == todos) )
  return nn_minus_a3

def findnn_plus(X,Y,Z):
  nn_plus_a3 = {}
  for todos in range(len(X)):

      UnSitio     = array([X[todos]-a3[0],Y[todos]-a3[1],Z[todos]-a3[2]])
      DataUnSitio = kron(ones(nsites),UnSitio)
      DataUnSitio = DataUnSitio.reshape(nsites,3)

      Dif = DataUnSitio - array([X,Y,Z]).T
      normas = []
      for n in range(len(Dif)):
          norm = sqrt(dot(Dif[n],Dif[n]))
          normas.append(norm)
      normas  = array(normas)
      logical = normas<1.44 # distancia en Ang para encontrar los vecinos

      nn_plus_a3[todos] = delete( logical.nonzero()[0], \
          where(logical.nonzero()[0] == todos) )
  return nn_plus_a3

def GetDATAplot(X,Y,Z,nn,nn_minus_a3,nn_plus_a3):
  DATA = [go.Scatter3d(x=X, y=Y, z=Z,mode='markers',showlegend=False)]

  for key in nn: # para cada sitio
      for n in nn[key]: # para cada vecino
          DATA.append( go.Scatter3d(x=[X[key],X[n]], y=[Y[key],Y[n]], z=[Z[key],Z[n]],mode='lines',line=dict(color='blue'),showlegend=False))

  for key in nn_minus_a3: # para cada sitio
      for n in nn_minus_a3[key]: # para cada vecino
          DATA.append( go.Scatter3d(x=[X[key]+a3[0],X[n]], y=[Y[key]+a3[1],Y[n]], z=[Z[key]+a3[2],Z[n]],mode='lines',line=dict(color='red'),showlegend=False) )

  for key in nn_plus_a3: # para cada sitio
      for n in nn_plus_a3[key]: # para cada vecino
          DATA.append( go.Scatter3d(x=[X[key]-a3[0],X[n]], y=[Y[key]-a3[1],Y[n]], z=[Z[key]-a3[2],Z[n]],mode='lines',line=dict(color='red'),showlegend=False) )

  annos = []
  # for n in range(len(X)):
  for n in range(len(X)):
    anno = dict(x=X[n],
                y=Y[n],
                z=Z[n],
                text=str(n),
                showarrow=False,
                arrowhead=0,
                font=dict(color='black'),
                ax=0,
                ay=0,
                bgcolor="white",
                opacity=0.2)

    annos.append(anno)

  return DATA,annos

In [4]:
nn = findnn(X,Y,Z)
nn_minus = findnn_minus(X,Y,Z)
nn_plus  = findnn_plus(X,Y,Z)

In [5]:
#KL1 = 365.0 #N/m
#KT1 = 245.0 #N/m
#KZ1 = 98.2 #N/m

KL1 = 1 #N/m
KT1 = 0 #N/m
KZ1 = 0 #N/m

Kbase = array([[KL1,0,0],
              [0,KT1,0],
              [0,0,KZ1]],dtype=complex)

def KGeneral(r1,r2):

  x1,y1,z1 = r1[0],r1[1],r1[2]
  x2,y2,z2 = r2[0],r2[1],r2[2]

  x12 = x2 - x1
  y12 = y2 - y1
  z12 = z2 - z1

  rij = [x12,y12,z12]

  zhat = [0,0,1]
  ρhat = [x1,y1,0]/LA.norm([x1,y1,0]).T
  θhat = np.cross(zhat,ρhat)

  M = array([θhat,zhat,ρhat]).T #Matriz de cambio de base

  φ = arcsin(dot(rij,ρhat)/LA.norm(rij)) #Angulo que hace Kx con el plano perpendicular a rho.

  T = array([[cos(φ) ,0.  ,-sin(φ)],
                [0.     ,1.  ,0],
                [sin(φ),0.  ,cos(φ)]],dtype = complex)

  w = rij - dot(rij,ρhat)*ρhat #Componente del vector dentro del plano

  xi = dot(w,θhat)
  yi = dot(w,zhat)

  θ = arcsin(yi/sqrt(xi**2 + yi**2))

  U = array([[cos(θ),-sin(θ),0],
              [sin(θ),cos(θ),0],
              [       0,      0,1]],dtype = complex)

  print(f"φ = {φ*180/pi}")
  print(f"θ = {θ*180/pi}")
  print(f"θhat = {θhat}")
  print(f"ρhat = {ρhat}")
  


  Krotplano = dot(inv(T),dot(Kbase,inv(T)))

  Krotpolar = dot(U,dot(Krotplano,inv(U)))

  print(f"Vector θhat rotado = {(dot(M,dot(T,dot(U,[1,0,0]))).real)}")
  print(f"Vector ρhat rotado = {(dot(M,dot(T,dot(U,[0,0,1])))).real}")
  print(f"Vector zhat rotado = {(dot(M,dot(T,dot(U,[0,1,0])))).real}")


  return dot(M,dot(Krotpolar,inv(M)))

In [6]:
DATA,annos = GetDATAplot(X,Y,Z,nn,nn_minus,nn_plus)
fig = go.Figure(data=DATA)
fig.update_layout(scene=dict(annotations=annos))
fig.show()

In [7]:
def HacerPrueba(r1,r2):
    KGeneral(r1,r2)
    print(KGeneral(r1,r2))
    print("--------------------------")
    print(KGeneral(r2,r1))
    return KGeneral(r1,r2)

In [8]:
rij = (r[43] - r[50])/LA.norm(r[50] - r[43])
a = HacerPrueba(r[43],r[50])

φ = -2.096366923617549
θ = -71.04187975084295
θhat = [ 0.78183148  0.6234898  -0.        ]
ρhat = [ 0.6234898  -0.78183148  0.        ]
Vector θhat rotado = [ 0.24641941  0.21171327 -0.94575629]
Vector ρhat rotado = [ 0.65167217 -0.75850074  0.        ]
Vector zhat rotado = [0.71735685 0.61632306 0.32487695]
φ = -2.096366923617549
θ = -71.04187975084295
θhat = [ 0.78183148  0.6234898  -0.        ]
ρhat = [ 0.6234898  -0.78183148  0.        ]
Vector θhat rotado = [ 0.24641941  0.21171327 -0.94575629]
Vector ρhat rotado = [ 0.65167217 -0.75850074  0.        ]
Vector zhat rotado = [0.71735685 0.61632306 0.32487695]
[[ 0.063909  +0.j  0.063909  +0.j -0.26145561+0.j]
 [ 0.04015668+0.j  0.04015668+0.j -0.16428345+0.j]
 [-0.21834386+0.j -0.21834386+0.j  0.89325808+0.j]]
--------------------------
φ = -2.096366923617541
θ = 71.04187975084295
θhat = [ 0.6234898   0.78183148 -0.        ]
ρhat = [ 0.78183148 -0.6234898   0.        ]
Vector θhat rotado = [0.19313053 0.26123865 0.94575629]
Vector ρ

In [9]:
rij

array([-0.23102156, -0.23102156,  0.94512331])

In [10]:
dot(a,rij)

array([-0.2766365 +0.j, -0.17382223+0.j,  0.94512331+0.j])